# Instructivo

Para que el segmentador funcione se requiere tener Python (version 3.7.7) instalado con las siguientes librerias:

1. pickle: para leer los archivos que almacenan los diccionarios. (version 0.7.5)
2. nltk: para preprocesamiento de las respuestas. (version 3.6.7)
3. spacy: para preprocesamiento de las respuestas. (version 3.2.1)
4. re: preprocesamiento de las respuestas. (version 2021.11.10)
5. KidESpell: corrector de ortografía infantil (version 0.0.2)

El segmentador y corrector (Segment_and_Correct_TM) corresponde a un objeto de python definido a continuación:

(Detalles de uso al final)

In [2]:
import pickle
import math as math
import spacy
import string
import nltk
import spacy
import re
from kidespell import SpellChecker
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk import word_tokenize
nlp = spacy.load('es_core_news_sm')
allstopwords = nlp.Defaults.stop_words

C:\Users\felip\anaconda3\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'es_core_news_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.4.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [3]:
class Segment_and_Correct_TM(object):
    
    def __init__(self,v1,v2,v3,v4,rae):
        self.v1 = v1
        self.v2 = v2
        self.v3 = v3
        self.v4 = v4
        self.rae = rae
        self.SC = SpellChecker()
        
    def segmentar(self,palabra,prints=False):
        palabra = palabra.replace(' ','')
        w1 = self.v1
        w2 = self.v2
        w3 = self.v3
        w4 = self.v4
        TOTAL = float(sum(w1.values()))
        test = -float("inf")
        l = len(palabra)
        p = [[0]]
        for j in range(1, l+1):
            imax = test
            p.append(p[-1]+[j])
            ascore = test
            atento = 0
            previus = ''
            best_ixs = None
            for i in range(j):
                ixs = p[i]+[j]
                if prints: print(ixs)
                last_segment = palabra[ixs[-2]:ixs[-1]]
                nscore = self.get_score_D(palabra, ixs)
                if atento == 1 and last_segment in w1.keys(): #Se activa comparación especial
                    pre_last_segment = palabra[ixs[-3]:ixs[-2]]
                    if pre_last_segment + last_segment ==  previus:
                        if prints: print('entro')
                        if len(ixs) >= 5:
                            if prints: print('comparacion cuatri-tri')
                            freq1 = self.get_frec4(palabra,ixs)
                            freq2 = self.get_frec3(palabra,best_ixs)
                            if prints: print('ixs: ',ixs,freq1,'-----','best_ixs:',best_ixs,freq2)
                            if freq2 < freq1:
                                imax = i
                                ascore = nscore
                                best_ixs = ixs
                                previus = last_segment
                        elif len(ixs) == 4:
                            if prints: print('comparacion tri-bi')
                            freq1 = self.get_frec3(palabra,ixs)
                            freq2 = self.get_frec2(palabra,best_ixs)
                            if prints: print('ixs: ',ixs,freq1,'-----','best_ixs:',best_ixs,freq2)
                            if freq2 < freq1:
                                imax = i
                                ascore = nscore
                                best_ixs = ixs
                                previus = last_segment
                        elif len(ixs) == 3:
                            if prints: print('comparacion bi-uni')
                            freq1 = self.get_frec2(palabra,ixs)
                            freq2 = self.get_frec1(palabra,best_ixs)
                            if prints: print('ixs: ',ixs,freq1,'-----','best_ixs:',best_ixs,freq2)
                            if freq2 < freq1:
                                imax = i
                                ascore = nscore
                                best_ixs = ixs
                                previus = last_segment

                        #se activa super comparacion
                    else:
                        if nscore>ascore:
                            imax = i
                            ascore = nscore
                else:
                    if nscore>ascore:
                        if last_segment in w1.keys():
                            atento = 1
                            best_ixs = ixs
                            previus = last_segment
                            imax = i
                            ascore = nscore
                        else:
                            atento = 0
                            previus = ''
                            imax = i
                            ascore = nscore
            p[j] = p[imax]+[j]
            #print('p final: ',p[j])
        p_min = p[l]

        return self.segment_by_pivots(palabra,p_min)
    
    def get_frec1(self,string,ixs):
        w1 = self.v1
        s1 = string[ixs[-2]:ixs[-1]]
        unigram = s1
        if unigram in w1.keys():
            return w1[unigram]
        else:
            return 0

    def get_frec2(self,string,ixs):
        w2 = self.v2
        s2 = string[ixs[-2]:ixs[-1]]
        s1 = string[ixs[-3]:ixs[-2]]
        bigram = s1+'<$>'+s2
        if bigram in w2.keys():
            return w2[bigram]
        else:
            return 0

    def get_frec3(self,string,ixs):
        w3 = self.v3
        s3 = string[ixs[-2]:ixs[-1]]
        s2 = string[ixs[-3]:ixs[-2]]
        s1 = string[ixs[-4]:ixs[-3]]
        trigram = s1+'<$>'+s2+'<$>'+s3
        if trigram in w3.keys():
            return w3[trigram]
        else:
            return 0

    def get_frec4(self,string,ixs):
        w4 = self.v4
        s4 = string[ixs[-2]:ixs[-1]]
        s3 = string[ixs[-3]:ixs[-2]]
        s2 = string[ixs[-4]:ixs[-3]]
        s1 = string[ixs[-5]:ixs[-4]]
        cuatrigram = s1+'<$>'+s2+'<$>'+s3+'<$>'+s4
        if cuatrigram in w4.keys():
            return w4[cuatrigram]
        else:
            return 0
        
    def segment_by_pivots(self,s,p):
        palabra_segmentada = ''
        if len(p)>2:
            pivote_past = 0
            for pivote in p[1:]:
                palabra_segmentada += s[pivote_past:pivote] + ' '
                pivote_past = pivote
            return palabra_segmentada[:-1]
        else:
            palabra_segmentada += s
            return palabra_segmentada
    
    def code(self,frase):
        lista_numeros = []
        lista_signos = []
        numero_formado = ''
        for caracter in frase:
            if caracter in ['0','1','2','3','4','5','6','7','8','9']:
                numero_formado += caracter
            if numero_formado != '' and caracter not in ['0','1','2','3','4','5','6','7','8','9','']:
                lista_numeros.append(numero_formado)
                numero_formado = ''
            if numero_formado != '' and caracter == frase[-1]:
                lista_numeros.append(numero_formado)
            if caracter in string.punctuation:
                lista_signos.append(caracter)
        return lista_numeros, lista_signos
    
    def normalize(self,s):
        replacements = (
            ("á", "a"),
            ("é", "e"),
            ("í", "i"),
            ("ó", "o"),
            ("ú", "u"),
        )
        for a, b in replacements:
            s = s.replace(a, b).replace(a.upper(), b.upper())
        return s

    def process_string(self,frase,no_num=False,replace_num=False,no_sw=False,no_punct=False,no_mayusq=False,lem=False,no_tildes=False):
        if no_num == True:
            if replace_num == True:
                pattern = r'[0-9]'
                frase = re.sub(pattern,'1',frase)
            else:
                pattern = r'[0-9]'
                frase = re.sub(pattern,'',frase)
        if no_punct == True:
            pattern = r'[^\w\s]'
            frase = re.sub(pattern, ' ! ', frase)
        frase = word_tokenize(frase)
        if no_mayusq == True:    
            frase = [w.lower() for w in frase]
        if no_sw == True:
            frase = [w for w in frase if w not in allstopwords]
        if no_tildes == True:
            frase_ = []
            for token in frase:
                frase_.append(normalize(token))
            frase = frase_
        if lem == True:
            frase_ = []
            frase = TreebankWordDetokenizer().detokenize(frase)
            frase = nlp(frase)
            for token in frase:
                frase_.append(token.lemma_)
            frase = frase_
        return frase, TreebankWordDetokenizer().detokenize(frase)
    
    def sec_ones_to_one_one(self,frase_token):
        new_frase_token = []
        for token in frase_token:
            if '11' in token:
                need_more = True
                while need_more != False:
                    token = token.replace('11','1')
                    if '11' not in token:
                        need_more = False
                        new_frase_token.append(token)
            else:
                new_frase_token.append(token)
        return new_frase_token
    
    def separador(self,frase_token):
        resultado = ''
        to_split = ''
        for token in frase_token:
            if token not in ['1','!']:
                to_split += token + ' '
                if token == frase_token[-1] and to_split != '':
                    resultado += self.segmentar(to_split)
            elif to_split != '' and token in ['1','!']:
                resultado += self.segmentar(to_split) + ' ' + token + ' '
                to_split = ''
            elif to_split == '' and token in ['1','!']:
                resultado += token + ' '   
        return resultado
    
    def final_corrector_Ngram(self,frase):
        code = self.code(frase)
        numeros = code[0]
        signos = code[1]
        frase = self.process_string(frase,no_num=True,replace_num=True,no_sw=False,no_punct=True,no_mayusq=True,lem=False,no_tildes=False)[0]
        resultado_final = ''
        frase = self.sec_ones_to_one_one(frase)
        frase_seg = self.separador(frase)
        frase_seg_correc = self.corrector_frase(frase_seg)
        frase_seg_correc_token = word_tokenize(frase_seg_correc)
        if numeros != [] or signos != []:
            i = 0
            p = 0
            for token in frase_seg_correc_token:
                if token == '!':
                    token = signos[p]
                    p += 1
                    resultado_final += token + ' '
                else:
                    if '1' in token:
                        lista = list(token)
                        for j in range(len(token)):
                            if lista[j] == '1':
                                lista[j] = numeros[i]
                                i += 1
                        token = ''.join(lista)
                        resultado_final += token + ' '
                    else:
                        resultado_final += token + ' '
        else:
            resultado_final += frase_seg_correc
        return resultado_final[:-1]
    
    def corrector(self,palabra):
        if '1' in palabra:
            return palabra
        else:
            sugerencias = self.SC.getSuggestionsForSentence(palabra)
            if sugerencias == {}:
                return palabra
            else:
                if sugerencias[palabra] == []:
                    return palabra
                else:
                    return sugerencias[palabra][0]
        
    def corrector_frase(self,frase):
        # frase debe ser la string con 1's y !'s
        frase_token = word_tokenize(frase)
        resultado = ''
        for token in frase_token:
            if '1' and '!' not in token and token not in self.rae.keys():
                resultado += self.corrector(token) + ' '
            else:
                resultado += token + ' '
        return resultado
    
    def cal_log_prob_dic(self,palabra_,prints=False):
        w1 = self.v1
        TOTAL = float(sum(w1.values()))
        if palabra_ in w1.keys():
            if w1[palabra_] <= 0:
                if prints: print('Aproximó')
                largo = float(len(palabra_))
                exp = math.log(5 * 10**302,TOTAL)
                if largo > exp:
                    largo = exp
                denominador = TOTAL**largo
                log_prob = math.log(10.0/denominador)
            else:
                if prints: print('No Aproximó')
                log_prob = math.log(float(w1[palabra_])/TOTAL)
        else:
            if prints: print('Aproximó')
            largo = float(len(palabra_))
            exp = math.log(5 * 10**302,TOTAL)
            if largo > exp:
                largo = exp
            denominador = TOTAL**largo
            log_prob = math.log(10.0/denominador)
        return log_prob

    def get_score_D(self,s,ixs):
        w1 = self.v1
        TOTAL = float(sum(w1.values()))
        ixs += [] if len(s) in ixs else [len(s)]
        tokens = []
        for k in range(len(ixs)-1):
            i, j = ixs[k], ixs[k+1]
            tokens.append(s[i:j])
        prob = 0
        for k in range(len(tokens)):
                sk = tokens[k]
                prob += self.cal_log_prob_dic(sk)
        return prob

    def TM_D(self,s):
        w1 = self.v1
        TOTAL = float(sum(w1.values()))
        test = -float("inf")
        l = len(s)
        p = [[0]]
        for j in range(1, l+1):
            imax = test
            p.append(p[-1]+[j])
            ascore = test
            for i in range(j):
                ixs = p[i]+[j]
                #print(ixs)
                nscore = self.get_score_D(s, ixs)    
                if nscore>ascore:
                    imax = i
                    ascore = nscore
            p[j] = p[imax]+[j]
            #print('p final: ',p[j])
        p_min = p[l]

        return self.segment_by_pivots(s,p_min)
    
    def separador_TM(self,frase_token):
        resultado = ''
        to_split = ''
        for token in frase_token:
            if token not in ['1','!']:
                to_split += token + ' '
                if token == frase_token[-1] and to_split != '':
                    resultado += self.TM_D(to_split)
            elif to_split != '' and token in ['1','!']:
                resultado += self.TM_D(to_split) + ' ' + token + ' '
                to_split = ''
            elif to_split == '' and token in ['1','!']:
                resultado += token + ' '   
        return resultado
    
    def final_corrector_TM(self,frase):
        code = self.code(frase)
        numeros = code[0]
        signos = code[1]
        frase = self.process_string(frase,no_num=True,replace_num=True,no_sw=False,no_punct=True,no_mayusq=True,lem=False,no_tildes=False)[0]
        resultado_final = ''
        frase = self.sec_ones_to_one_one(frase)
        frase_seg = self.separador_TM(frase)
        frase_seg_correc = self.corrector_frase(frase_seg)
        frase_seg_correc_token = word_tokenize(frase_seg_correc)
        if numeros != [] or signos != []:
            i = 0
            p = 0
            for token in frase_seg_correc_token:
                if token == '!':
                    token = signos[p]
                    p += 1
                    resultado_final += token + ' '
                else:
                    if '1' in token:
                        lista = list(token)
                        for j in range(len(token)):
                            if lista[j] == '1':
                                lista[j] = numeros[i]
                                i += 1
                        token = ''.join(lista)
                        resultado_final += token + ' '
                    else:
                        resultado_final += token + ' '
        else:
            resultado_final += frase_seg_correc
        return resultado_final[:-1]

# Ejemplo de uso:

Se deben cargar los 5 diccionarios a utilizar:

1. Unigramas (w1)
2. Bigramas (w2)
3. Trigramas (w3)
4. Cuatrigramas (w4)
5. Versión palabras sin tildes de diccionario de Unigramas (CREA_tildes)

Para inicializar el segmentador con corrector KidESpell escribimos:

`corrector = Segment_and_Correct_TM(w1_rae,w2,w3,w4,CREA_tildes)`

Luego podemos segmentar y corregir con los métodos:

1. final_corrector_Ngram
2. final_corrector_TM

Donde el primero corresponde al algoritmo con la implementación de N-Gramas y el segundo corresponde a la implementación del algortimo original. Para utilizarlos:

1. `corrector.final_corrector_Ngram(frase)`
2. `corrector.final_corrector_TM(frase)`

Siendo ''frase'' el texto a procesar en formato string.

In [4]:
# CARGAMOS LOS DICCIONARIOS

# No hay una dirección del archivo pues los archivos en mi ordenador están en la misma 
# carpeta que este archivo notebook.
    
with open('w1_rae.pkl', 'rb') as handle:
    w1_rae = pickle.load(handle)
    
with open('w2.pkl', 'rb') as handle:
    w2 = pickle.load(handle)
    
with open('w3.pkl', 'rb') as handle:
    w3 = pickle.load(handle)
    
with open('w4.pkl', 'rb') as handle:
    w4 = pickle.load(handle)
    
with open('rae_tildes.pkl', 'rb') as handle:
    CREA_tildes = pickle.load(handle)

In [7]:
# INICIALIZAMOS
corrector = Segment_and_Correct_TM(w1_rae,w2,w3,w4,CREA_tildes)

In [11]:
%%time
r = 'no estoi deaquerdo por que 8mas 8 y mas 8 son16'
a = corrector.final_corrector_Ngram(r)
print('N-Gramas: ' + a)

N-Gramas: no estoy de acuerdo porque 8 mas 8 y mas 8 son16
Wall time: 5.86 s


In [25]:
len(r)

86

In [12]:
%%time
b = corrector.final_corrector_TM(r)
print('TM_original: ' + b)

TM_original: no estoy de acuerdo por que 8 mas 8 y mas 8 son 16
Wall time: 12.4 s


In [15]:
import pandas as pd

In [18]:
test = pd.read_excel("D:\github/coherence-prediction/data/test_task_C1.xlsx")
test

,id,Q_id,Q,A,label
0,85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0
1,64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0
2,84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0
3,88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1
4,51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1
...,...,...,...,...,...
672,54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0
673,87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0
674,73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1
675,95242,5093,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4...",0


In [19]:
import time
from unidecode import unidecode

In [32]:
%%time
corrected = []
for ix in test.index:
    s = time.time()
    r = test.loc[ix]["A"]
    r = unidecode(str(r).replace("\n", ""))
    b = corrector.final_corrector_TM(r) if len(r) <= 85 else corrector.final_corrector_Ngram(r)
    corrected.append([ix, b])
    proc_time = time.time()-s
    print(f"Processing time was {proc_time} seconds")
    print(100*(ix+1)/test.shape[0], ix, b)

Processing time was 61.709574699401855 seconds
0.14771048744460857 0 haber las cajas porque las cajas chicas eran los conejos negro y la grande era blanco
Processing time was 160.37047147750854 seconds
0.29542097488921715 1 el proceso de division de una celula en dos celulas hijas recibe el nombre de mitosis
Processing time was 7.812733888626099 seconds
0.4431314623338257 2 debes practicar los ejerccios
Processing time was 8.474916219711304 seconds
0.5908419497784343 3 no se porque no fui . a donde fueron ustedes
Processing time was 1.867795705795288 seconds
0.7385524372230429 4 las sopaipillas
Processing time was 0.4850001335144043 seconds
0.8862629246676514 5 no se no se
Processing time was 2.1508901119232178 seconds
1.03397341211226 6 no por que me da 673
Processing time was 3.7107656002044678 seconds
1.1816838995568686 7 malo por que le digo 673 + 673 + 673 = 2 . 019
Processing time was 0.0 seconds
1.329394387001477 8 ,
Processing time was 0.0010454654693603516 seconds
1.4771048744

Processing time was 1.0407733917236328 seconds
10.635155096011816 71 no vine tio
Processing time was 1.4370338916778564 seconds
10.782865583456426 72 esta mal son 18 metros
Processing time was 29.096975088119507 seconds
10.930576070901035 73 juan tiene 24 lapices minas y los quiere repartir en sus 4 amigos ? cuantos lapices minas r recibirá cada unocuantos lapices minas r recibirá cada uno ? recibir an 6 cada uno
Processing time was 11.954323768615723 seconds
11.078286558345642 74 que hay que aprender y estudiar
Processing time was 31.928061485290527 seconds
11.225997045790251 75 todo se puedo amigito todo en la vida se puede
Processing time was 17.42029857635498 seconds
11.37370753323486 76 las en ora juana tenia 245 gallinas y el se honor pedro leregalo 123 ? cuantas gallinas tiene en total juana ? respuestas : 358 .
Processing time was 0.20586276054382324 seconds
11.521418020679468 77 un 7 o un 6
Processing time was 110.37320566177368 seconds
11.669128508124077 78 el doble de lo que

Processing time was 1.052673101425171 seconds
20.38404726735598 137 que se no se
Processing time was 6.901337385177612 seconds
20.53175775480059 138 esta mal por que son 3x673 = 2019 en total
Processing time was 4.661167621612549 seconds
20.6794682422452 139 esta mal por que eran 630
Processing time was 42.200724601745605 seconds
20.82717872968981 140 claudia recolecto 74 banderas y para resolver use la suma
Processing time was 0.0009989738464355469 seconds
20.974889217134418 141 7
Processing time was 0.45534586906433105 seconds
21.122599704579024 142 el . cerebro .
Processing time was 0.3879978656768799 seconds
21.270310192023633 143 no fui .
Processing time was 0.0009999275207519531 seconds
21.418020679468242 144 .
Processing time was 30.56598949432373 seconds
21.56573116691285 145 porque al dividir dos potencias iguales el resultado es 1 y eso es equivalente a la potencia 0
Processing time was 0.3916189670562744 seconds
21.71344165435746 146 esta mal
Processing time was 19.641183376

Processing time was 6.495995044708252 seconds
31.16691285081241 210 cuantas pautas a y en total
Processing time was 0.38926196098327637 seconds
31.314623338257014 211 17 45 minutos
Processing time was 14.414961576461792 seconds
31.462333825701624 212 que no se puede separar o dividir
Processing time was 8.69729208946228 seconds
31.610044313146233 213 si porque dice que guarda 3 chocolates en 2 cajistas y 3 mas 3 es 6
Processing time was 29.70954203605652 seconds
31.757754800590842 214 no esta correcto le tenia que pasar de vuelto 1290
Processing time was 60.942830085754395 seconds
31.90546528803545 215 el sistema circulatorio hace que lo que comemosel sistema circulatorio hace que lo que comemos sea digerido y . sin ese sistema no se digereria lo que comemos
Processing time was 46.545777797698975 seconds
32.05317577548006 216 el cerebro es como el musculo , si practicas mas roman aquiles mas se te fortalecer
Processing time was 74.64584016799927 seconds
32.20088626292467 217 si se reno

Processing time was 2.5029103755950928 seconds
41.3589364844904 279 esta mal por que 4 * 7 son 28
Processing time was 21.682927131652832 seconds
41.506646971935005 280 son dos que todo y llegue al resultado
Processing time was 3.130871057510376 seconds
41.65435745937962 281 esta mal porque era 135
Processing time was 37.22010016441345 seconds
41.802067946824224 282 lo que dice su primo no es correcto por que son 3
Processing time was 9.053102493286133 seconds
41.949778434268836 283 no le alcanza porque ai 20 personas y son 16 hamburguesa
Processing time was 0.0 seconds
42.09748892171344 284 1126
Processing time was 106.62189316749573 seconds
42.24519940915805 285 que estudie en su casa y que le ayudara en las matematica entretenida
Processing time was 0.11799860000610352 seconds
42.39290989660266 286 24 de 24 a 2 y 2 a 4
Processing time was 2.3188202381134033 seconds
42.540620384047266 287 esta bien porque 3 + 3 son 6
Processing time was 6.22963285446167 seconds
42.68833087149188 288 e

Processing time was 95.02572464942932 seconds
51.84638109305761 350 esta mal lo que le dijo la amiga sobre los dulcesesta mal lo que le dijo la amiga sobre los dulces que le regalo porque en realidad regalo 14 dulces
Processing time was 0.0009975433349609375 seconds
51.994091580502214 351 16
Processing time was 2.1023104190826416 seconds
52.14180206794683 352 el resultado es . digamelo
Processing time was 10.648667812347412 seconds
52.28951255539143 353 si por que no existe desimal
Processing time was 4.100175857543945 seconds
52.43722304283604 354 esta mal porque mi de 90 grados
Processing time was 8.011644840240479 seconds
52.58493353028065 355 maicol quiere repartir sus 48 dulces en sus 6 amigos
Processing time was 7.356168508529663 seconds
52.732644017725256 356 esta mal por que le quedan 14
Processing time was 70.52986025810242 seconds
52.88035450516987 357 camila tiene 56 m une casi y debe de repartirlas en 7 amigasamigas que invito a su casa ? cuantas m une casi tendra cada una 

Processing time was 4.232059478759766 seconds
61.595273264401776 416 esta malo por que 7x5 = 5
Processing time was 24.792322635650635 seconds
61.74298375184638 417 juan compro 4 cajas de huevos cajas de huevos y en cada cajas venian 8 huevos huevos tenia 16 huevos
Processing time was 18.41584348678589 seconds
61.89069423929099 418 yo opino que su hermana tiene razón
Processing time was 41.1137900352478 seconds
62.0384047267356 419 en el azul porque en el grafico se ve mas aumento .
Processing time was 26.605793952941895 seconds
62.186115214180205 420 felipe tiene 9 caramelos y lo tiene web rrepartir en 3 amigos cuantos caramelos lecorresponde a cada amigo
Processing time was 4.152083158493042 seconds
62.33382570162482 421 que no se porte mal
Processing time was 2.8126816749572754 seconds
62.48153618906942 422 me da de resultado 332
Processing time was 29.82823085784912 seconds
62.62924667651403 423 diego tiene razon por que multiplico y le dio 24 por que conto los cubos
Processing time

Processing time was 38.736403465270996 seconds
71.93500738552437 486 jose tenia 245 tablas y 2123 palos y todo eso lo ben dio y cuanto es en total
Processing time was 25.473417282104492 seconds
72.08271787296898 487 esta mal porque todo numero multiplicado por 0 es 0
Processing time was 39.30758857727051 seconds
72.23042836041358 488 diego llega hace casadiego llega hace casa alas 18 : 25 hrs . despues de llegar al cine y ver la pelicula haces ahora llego a su casa
Processing time was 19.268428087234497 seconds
72.3781388478582 489 le faltan 4 hamburguesa y multiplique 4x4 es igual a 16 hamburguesas
Processing time was 0.05399894714355469 seconds
72.52584933530281 490 un 70
Processing time was 15.163038730621338 seconds
72.67355982274742 491 no ese hombre de mas es do mil diez
Processing time was 5.743102312088013 seconds
72.82127031019202 492 esta mal porque es 15 paginas paginas no son 17 paginas
Processing time was 0.3800220489501953 seconds
72.96898079763663 493 unidad
Processing t

Processing time was 68.30425763130188 seconds
82.27474150664698 556 esta mal porque es 5 / 7 en el resultado la pregunta de alonso esta mal porque dice que se comio alonso 2 / 7 y se comio 3 / 7 y se comio dos de los tres de los chocolate
Processing time was 12.209162473678589 seconds
82.42245199409157 557 yo opino que esta bien por que 6 x 64 es = 4
Processing time was 2.2572474479675293 seconds
82.57016248153619 558 esta mal por es 520
Processing time was 52.31369495391846 seconds
82.7178729689808 559 se mover a al mayor porque el mayor tendra mas nutrientes
Processing time was 0.15482592582702637 seconds
82.86558345642541 560 no se
Processing time was 2.672147035598755 seconds
83.01329394387001 561 esta jugando bien
Processing time was 4.713252544403076 seconds
83.16100443131462 562 yo le pongo onu siete
Processing time was 5.419058322906494 seconds
83.30871491875924 563 que estudie y sea mejor
Processing time was 2.401698589324951 seconds
83.45642540620383 564 no se me ocurre
Proce

Processing time was 1.7192566394805908 seconds
93.0576070901034 629 naranja 245 y le dieron
Processing time was 49.35893654823303 seconds
93.205317577548 630 donde hayan mas nutrientes para tener un mejor desarrollo
Processing time was 0.101318359375 seconds
93.35302806499261 631 la n
Processing time was 0.15900182723999023 seconds
93.50073855243723 632 no se
Processing time was 0.6397302150726318 seconds
93.64844903988183 633 ensenarle
Processing time was 19.406731843948364 seconds
93.79615952732644 634 esta mal esta mal por en total son 51 lo que dice el alumno esta mal
Processing time was 17.327297925949097 seconds
93.94387001477105 635 la reunion de martin empezó 15 : 20 y termino a las 17 : 30 cuantos minutos se demoro
Processing time was 4.174865961074829 seconds
94.09158050221566 636 porfia no se ajajjaajj
Processing time was 20.543158292770386 seconds
94.23929098966026 637 no esta correcto por que yo sume 3 veces 673 y me dio 2 . 019 y el vendedor esta mal
Processing time was 1

In [33]:
import pickle

In [35]:
seg = test.copy()
seg["TM"] = [x[1] for x in corrected]
seg

,id,Q_id,Q,A,label,TM
0,85815,4581,¿Qué aprendiste de la actividad del viernes?,ha ver las cajas porque las cajas chicas eran ...,0,haber las cajas porque las cajas chicas eran l...
1,64812,3535,¿Cómo creen ustedes que se reproducen las célu...,El proceso de división de una célula en dos cé...,0,el proceso de division de una celula en dos ce...
2,84148,4491,¿ Qué frase para mejorar le darías tú a un am...,debes practicar los ejerccios,0,debes practicar los ejerccios
3,88565,4734,¿Qué aprendiste en la actividad del día viernes?,Nose porque no fui. A donde fueron ustedes😛😛😛😛😜,1,no se porque no fui . a donde fueron ustedes
4,51688,2944,Diego compró 2 sopaipillas a $120 cada una. Pa...,las sopaipillas,1,las sopaipillas
...,...,...,...,...,...,...
672,54685,3109,Lorena dice que al multiplicar 3x0 le da como...,estamal poreque es 0,0,esta mal porqué es 0
673,87324,4665,Alicia comió 2/9 de un chocolate. Ella dice qu...,esta mal Alicia por que la cantidad es 7/8 de...,0,esta mal alicia por que la cantidad es 7 / 8 d...
674,73798,3946,El cuarto básico visitará el zoológico. Si son...,nose\n,1,no se
675,95242,5093,Durante la sesión de conecta ideas María Jesús...,"es correcto, porque yo lo \nmultiplique por 4...",0,"es correcto , porque yo lo multiplique por 4 l..."


In [36]:
seg.to_excel("seg_TM_test_v1.xlsx")

In [40]:
r = "que se es fuese"
corrector.final_corrector_TM(r), corrector.final_corrector_Ngram(r)

('que se es fuese', 'que se esfuese')